In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import json
import networkx as nx

In [2]:
# data_path = './Tiktok Data'
data_path = '../../../../Library/CloudStorage/OneDrive-DrexelUniversity/Youth_Substance_Use/Tiktok Data/N-hits frames/'
# is directory?
print(os.path.isdir(data_path))

True


In [ ]:
# for file in os.listdir(data_path):
#     if file == '.DS_Store':
#         continue
#     print(os.listdir(os.path.join(data_path, file)))

In [5]:
# all_hastags = pd.DataFrame()

# for a in os.listdir(data_path):
#     if a == '.DS_Store':
#         continue
#     b = os.path.join(data_path, a)
#     # print(b)
#     for file in os.listdir(b):
#         if file == '.DS_Store':
#             continue
#         c = os.path.join(b, file)
#         # print(c)
#         for sub_file in os.listdir(c):
#             if sub_file.endswith('cooccurence.csv'):
#                 data = pd.read_csv(os.path.join(c, sub_file))
#                 # get source hashtag from filname
#                 source_hashtag = sub_file.split('_')[0]
#                 data['source_hashtag'] = source_hashtag
#                 all_hastags = pd.concat([all_hastags, data])
#                 break

NotADirectoryError: [Errno 20] Not a directory: '../../../../Library/CloudStorage/OneDrive-DrexelUniversity/Youth_Substance_Use/Tiktok Data/N-hits frames/combined_4_hits_frame.csv'

In [5]:
# all_hastags.nunique()

word              86318
count               316
source_hashtag       54
dtype: int64

In [6]:
# export word to csv
# all_hastags["word"].to_csv("all_hashtags.csv", index=False)

In [13]:
# # export as edgelist for gephi
# all_hastags = all_hastags[['source_hashtag', 'word', 'count']]
# all_hastags = all_hastags.rename(columns={'source_hashtag': 'Source', 'word': 'Target', 'count': 'Weight'})
# all_hastags.to_csv('hashtag_cooccurence_edgelist_20240521.csv', index=False)


In [14]:
# filter
# all_hastags_filtered = all_hastags[all_hastags['Weight'] > 5]

In [15]:
# remove self loops
# all_hastags_filtered = all_hastags_filtered[all_hastags_filtered['Source'] != all_hastags_filtered['Target']]

In [16]:
# all_hastags_filtered.to_csv('hashtag_cooccurence_edgelist_filtered_20240521.csv', index=False)

---
Looking at the new data from jo 7/2/2024 which is just videos that have at least 2, 3, or 4 source hashtags

In [3]:
double_hits = pd.read_csv(data_path + 'combined_double_hits_frame_no_duplicates.csv')

In [4]:
double_hits.columns

Index(['uniqueID', 'Video ID', 'Video URL', 'hashtags', 'collectCount',
       'commentCount', 'diggCount', 'playCount', 'shareCount', 'file_path',
       'matching_hashtags'],
      dtype='object')

In [9]:
double_hits.hashtags.head()

0    ['mentalhealth', '5', 'five', 'coke', '8ball',...
1    ['mentalhealth', 'crystal', 'speed', 'crank', ...
2    ['mentalhealth', 'coke', 'snow', '8ball', 'spe...
3    ['mentalhealth', 'trap', 'traphouse', 'house',...
4    ['mentalhealth', 'adhd', 'adhdtiktok', 'coke',...
Name: hashtags, dtype: object

In [10]:
type(double_hits.matching_hashtags[0])

str

In [5]:
from collections import defaultdict
import itertools
import ast 
import json

edge_weights = defaultdict(int)

for i, row in double_hits.iterrows():
    hashtags = ast.literal_eval(row['hashtags'])
    matching = ast.literal_eval(row['matching_hashtags'])
    # create all possible pairs
    for source in matching:
        for target in hashtags:
            if source != target and target != '':
                edge_weights[(source, target)] += 1
    # for pair in itertools.combinations(matching, 2):
    #     sorted_pair = tuple(sorted(pair))
    #     edge_weights[sorted_pair] += 1

edges_df = pd.DataFrame(edge_weights.items(), columns=['pair', 'weight'])
edges_df[['source', 'target']] = pd.DataFrame(edges_df['pair'].tolist(), index=edges_df.index)
edges_df = edges_df.drop(columns=['pair'])

In [12]:
edges_df.head()

,weight,source,target
0,12,nicotine,mentalhealth
1,1,nicotine,5
2,1,nicotine,five
3,1,nicotine,coke
4,1,nicotine,8ball


In [6]:
print("number of unique nodes: ", len(set(edges_df.source.tolist() + edges_df.target.tolist())))

number of unique nodes:  152278


In [14]:
edges_df.to_csv('../data/double_hits_edges_no_dupes.csv', index=False)   

In [7]:
# redo with triple hits
triple_hits = pd.read_csv(data_path + 'combined_triple_hits_frame_no_duplicates.csv')


edge_weights = defaultdict(int)

for i, row in triple_hits.iterrows():
    hashtags = ast.literal_eval(row['hashtags'])
    matching = ast.literal_eval(row['matching_hashtags'])
    # create all possible pairs
    for source in matching:
        for target in hashtags:
            if source != target and target != '':
                edge_weights[(source, target)] += 1
    # for pair in itertools.combinations(matching, 2):
    #     sorted_pair = tuple(sorted(pair))
    #     edge_weights[sorted_pair] += 1

edges_df = pd.DataFrame(edge_weights.items(), columns=['pair', 'weight'])
edges_df[['source', 'target']] = pd.DataFrame(edges_df['pair'].tolist(), index=edges_df.index)
edges_df = edges_df.drop(columns=['pair'])

In [16]:
edges_df.head()

,weight,source,target
0,7,nicotine,mentalhealth
1,1,nicotine,5
2,1,nicotine,five
3,1,nicotine,coke
4,1,nicotine,8ball


In [9]:
print("number of unique nodes: ", len(set(edges_df.source.tolist() + edges_df.target.tolist())))

number of unique nodes:  23012


In [17]:
edges_df.to_csv('../data/triple_hits_edges_no_dupes.csv', index=False)

In [10]:
# redo for quadruple hits
quadruple_hits = pd.read_csv(data_path + 'combined_4_hits_frame_no_duplicates.csv')


In [11]:
edge_weights = defaultdict(int)

for i, row in quadruple_hits.iterrows():
    hashtags = ast.literal_eval(row['hashtags'])
    matching = ast.literal_eval(row['matching_hashtags'])
    # create all possible pairs
    for source in matching:
        for target in hashtags:
            if source != target and target != '':
                edge_weights[(source, target)] += 1
    # for pair in itertools.combinations(matching, 2):
    #     sorted_pair = tuple(sorted(pair))
    #     edge_weights[sorted_pair] += 1

edges_df = pd.DataFrame(edge_weights.items(), columns=['pair', 'weight'])
edges_df[['source', 'target']] = pd.DataFrame(edges_df['pair'].tolist(), index=edges_df.index)
edges_df = edges_df.drop(columns=['pair'])

In [20]:
edges_df.head()

,weight,source,target
0,6,nicotine,mentalhealth
1,1,nicotine,5
2,1,nicotine,five
3,1,nicotine,coke
4,1,nicotine,8ball


In [12]:
print("number of unique nodes: ", len(set(edges_df.source.tolist() + edges_df.target.tolist())))

number of unique nodes:  14154


In [21]:
edges_df.to_csv('../data/quadruple_hits_edges_no_dupes.csv', index=False)